In [ ]:
using BenchmarkTools

immutable Arc
    from::Int
    to::Int
    cost::Int
end

In [ ]:
# Function to read from file and get adj_list AND maximum Cost

function get_list()    
    data=readcsv("DijkNet_2.csv",Int)
    n=maximum(data[:,1:2])
    maxCost=maximum(data[:,3])
    adjL=Array(Array{Arc},n)
    for i=1:n adjL[i]=[] end

    for i =1:size(data,1)
        a,b,c=data[i,:]
        push!(adjL[a],Arc(a,b,c))
    end
    adjL,maxCost
end

In [ ]:
#----------  Lecture's Code of Dijkstra
#----------  O(n^2)
function simpleShortestPath(s,adjL)
    n=length(adjL)
    pred=zeros(Int,n)
    S=Int[]
    S̄=[1:n...]
    d=ones(Int,n)*typemax(Int) # Some big number 
    d[s]=0
    while length(S)<n
        i=S̄[indmin(d[S̄])]
        push!(S,i)
        deleteat!(S̄,findfirst(S̄.==i))
        for arc in adjL[i]
            j=arc.to
            new_d=d[i]+arc.cost
            if d[j]>new_d
                d[j]=new_d
                pred[j]=i
            end
        end
    end
    pred,d
end

In [ ]:
function find_min(d,S̄)
    min_val=typemax(Int)
    best_indx=-1
    for i in S̄
        if (d[i]<min_val)
            min_val= d[i]
            best_indx=i
        end
    end
    best_indx
end

In [ ]:
function simpleShortestPathFM(s,adjL)
    n=length(adjL) #O(1)
    pred=zeros(Int,n)  #O(n)
    S=Int[]    # O(1)
    S̄=[1:n...]  #O(n)
    d=ones(Int,n)*typemax(Int) # Some big number  O(n)
    d[s]=0 #(O(1))
    while length(S)<n   # The loop will repeat n times
        i=find_min(d,S̄) #O(n)
        push!(S,i) #O(1)
        deleteat!(S̄,findfirst(S̄.==i)) #If S is an array or list, this is O(n)
        for arc in adjL[i]
            j=arc.to
            new_d=d[i]+arc.cost
            if d[j]>new_d
                d[j]=new_d
                pred[j]=i
            end
        end
    end
    pred, d
end

In [ ]:
#Mention the errors in this implementation
function dial(s,adjL,maxCost)
    n=length(adjL)   #O(1)
    pred=zeros(Int,n) #O(n)
    buckets_size=(n*maxCost)+1 #O(1)
    d=Array(Array{Int}, buckets_size) 
    for i=1:buckets_size d[i]=[] end
    
    for i=1:n push!(d[end],i) end
    push!(d[1],s)
    
    distance_array=ones(n)*typemax(Int) # Some big number  O(n)
    distance_array[s]=0 #(O(1))
    
    for a =1:buckets_size #O(nc+1)
        
        if !isempty(d[a]) #O(1)
            i= pop!(d[a])
            for arc in adjL[i] #O(m)
                j=arc.to
                new_d=a + arc.cost  
                if distance_array[j] > new_d 
                    old_bucket=d[distance_array[j]+1]
                    distance_array[j]=new_d 
                    push!(d[new_d+1],j)
                    pred[j]=i
                end
            end
        end
    end
    pred, distance_array
end

In [ ]:
# Now doing my version of dial where I pop the distance nodes 
function dial2(s,adjL,maxCost)
    n=length(adjL)   #O(1)
    pred=zeros(Int,n) #O(n)
    buckets_size=(n*maxCost)+1 #O(1)
    d=Array(Array{Int}, buckets_size) 
    for i=1:buckets_size d[i]=[] end #Allocate some extra space
    
    for i=1:n 
        i==s || push!(d[end],i)  #FIXED removed 
    end 
    push!(d[1],s)
    #FIXED: now it of type Int also not the maximum type value
    distance_array=ones(Int,n)*(n*maxCost) # Some big number  O(n) 
    distance_array[s]=0 #(O(1))
    
    
    for a =1:buckets_size #O(nc+1)
        while !isempty(d[a]) #O(1)   NEWFIX
            i= pop!(d[a])
            for arc in adjL[i] #O(m)
                j=arc.to
                new_d=a-1 + arc.cost  #NEWFIX
                if distance_array[j] > new_d 
                    old_bucket=d[distance_array[j]+1]
                    deleteat!(old_bucket,findfirst(old_bucket,j))# FIXED Removing from bucket
                    distance_array[j]=new_d 
                    push!(d[new_d+1],j)
                    pred[j]=i
                end
            end
        end
    end
    pred, distance_array
end

In [ ]:
adjL,maxCost= get_list()  #FIXME: Always get elapsed time after doing one execution to eleminate compile time
s= 1;

println(simpleShortestPath(s,adjL))
println(simpleShortestPathFM(s,adjL))
println(dial(s,adjL,maxCost))

In [ ]:
@time simpleShortestPath(s,adjL);
@time simpleShortestPathFM(s,adjL);
@time dial2(s,adjL,maxCost);

In [ ]:
t1= @benchmark simpleShortestPath(s,adjL)
t2= @benchmark simpleShortestPathFM(1,adjL)
t3= @benchmark dial2(s,adjL,maxCost)
println( round(Int,mean(t1.times)/1000) )
println( round(Int,mean(t2.times)/1000) )
println( round(Int,mean(t3.times)/1000) )

In [ ]:
using StatsBase
# First lets make a graph generator 
function graphGen(num_nodes::Int,min_arcs::Int,max_arcs::Int,max_dist::Int)
    adjL=Array{Array{Arc}}(num_nodes)
    for i=1:num_nodes
        destinations=sample(1:num_nodes,rand(min_arcs:max_arcs),replace=false)
        #         destinations=randperm(num_nodes)[1:rand(min_arcs:max_arcs)] #too slow
        adjL[i]=[Arc(i,dest,rand(1:max_dist)) for dest in destinations]
    end
    adjL
end

In [ ]:
using StatsBase
sample(1:10,3)

In [ ]:
 maxCost=12
 aL=graphGen(100000,7,12,maxCost);

In [ ]:
@time simpleShortestPath(s,aL);
@time dial2(s,aL,maxCost);

In [ ]:
function compare(number_of_iterations,step_nodes,step_edges,start_node,maxCost)
    dials_running_times=zeros(number_of_iterations)
    Dijkstra_running_times=zeros(number_of_iterations)
    networkSize=zeros(number_of_iterations)
    nodes=100000
    min_out_degree = 7#0
    max_out_degree = 12#2
    for i = 1:number_of_iterations
        #nodes += step_nodes
        #min_out_degree += step_edges
        #max_out_degree += step_edges
        maxCost += step_nodes
        
        aL=graphGen(nodes,min_out_degree,max_out_degree,maxCost)
        
        networkSize[i]=nodes  

        t1=@timed dial2(start_node,aL,maxCost)
        t2=@timed simpleShortestPath(start_node,aL)
        
        
        dials_running_times[i] =t1[3]   
        Dijkstra_running_times[i] =t2[3] 
    end
    networkSize, dials_running_times,Dijkstra_running_times
end

In [ ]:
using PyPlot

In [ ]:
number_of_iterations =5
step_nodes =100
step_edges =40
start_node =1
maxCost = 12
networkSize, dials_running_times,dijkstra_running_times= compare(number_of_iterations,step_nodes,step_edges,start_node,maxCost)


plot(networkSize, dials_running_times, marker="*", label="dials time")
plot(networkSize, dijkstra_running_times, marker="*", label="dijkstra time")
xlabel("Graph Size")
ylabel("Time (s)")
legend(loc="upper right",fancybox="true")
title("Algorithms Performance Comparison")

In [ ]:
number_of_iterations =5
step_nodes =10
step_edges =40
start_node =1
maxCost = 3
networkSize, dials_running_times,dijkstra_running_times= compare(number_of_iterations,step_nodes,step_edges,start_node,maxCost)


plot(networkSize, dials_running_times, marker="*", label="dials time")
plot(networkSize, dijkstra_running_times, marker="*", label="dijkstra time")
xlabel("Graph Size")
ylabel("Time (s)")
legend(loc="upper right",fancybox="true")
title("Algorithms Performance Comparison")